In [1]:
import boto3
import pandas as pd
from io import StringIO

# create s3 client
s3 = boto3.client('s3')

# specify the bucket & folder you'd like to interact with
# will be using the cyber data as an example
name = "data-analyst-job-east"
file = "raw/gsearch_jobs.csv"

single_object = s3.get_object(Bucket=name, Key=file)
single_df = pd.read_csv(single_object['Body'])

/Users/pcordero/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


NoCredentialsError: Unable to locate credentials

In [ ]:
single_df.head()

In [ ]:
single_df['company_name'].value_counts()


In [ ]:
# 1) Drop staffing agencies (keywords to Staffing, Hire, Global, etc)

filtered_df = single_df[~single_df['company_name'].isin(['Upwork', 'Talentify.io'])]

company_counts = filtered_df['company_name'].value_counts()

In [ ]:
filtered_df['company_name'].value_counts()

In [ ]:
filtered_df.columns

In [ ]:
# 2) Most important columns: description_token, salary_standardized, title, date_time   (drop everything else)

selected_columns_df = filtered_df[['description_tokens', 'salary_standardized', 'title', 'date_time','company_name']]
selected_columns_df

In [ ]:
# 3) drop 39,000 missing rows of salary_standardized

selected_columns_df.isnull().sum() 

In [ ]:
selected_columns_df = selected_columns_df.dropna(subset=['salary_standardized'])
selected_columns_df

In [ ]:
# 4) join in the missing data
import boto3
import pandas as pd
from io import StringIO

# create s3 client
s3 = boto3.client('s3')

# specify the bucket & folder you'd like to interact with
# will be using the cyber data as an example
name = "data-analyst-job-east"
file = "raw/LinkedIn_Clean_List.csv"

Second_object = s3.get_object(Bucket=name, Key=file)
Second_df = pd.read_csv(Second_object['Body'])

In [ ]:
Second_df.head()

In [ ]:
Second_df = Second_df.rename(columns={'Company': 'company_name'})

In [ ]:
Second_df = Second_df.rename(columns={'Title': 'title'})

In [ ]:
#https://medium.com/codex/data-cleaning-using-pandas-c65e20a84c4b

In [ ]:
Second_df

In [ ]:
#Second_df = Second_df.rename(columns={'average_salary': 'salary_standardized'})
Second_df = filtered_df[['title', 'company_name']]
Second_df

In [ ]:
# Combine the dataframes
combined_df = pd.concat([selected_columns_df, Second_df], ignore_index=True)

In [ ]:
combined_df

In [ ]:
# I still need to EDA Analysis for the Combined data
combined_df.to_csv('output.csv', index=False)

In [ ]:
# 5) Load data back into s3 bucket
import boto3

# open client
client = boto3.client('s3')

# some initial variables
bucket_name = "data-analyst-job-east"
object_key = "transformed/combined_data.csv"

# open the file in binary format, and save into the var 'data'
with open("output.csv", "rb") as f:
    data = f.read()

### ADDING OBJECTS TO A BUCKET ###
response = client.put_object(
    Body=data,
    Bucket=bucket_name,
    Key=object_key
)